In [15]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils

In [60]:
#read in the flattened dataframes with the labels
MSFT_df = pd.read_csv("/Users/fahad/project_repo/data/external/MSFT_flat.csv")
AAPL_df = pd.read_csv("/Users/fahad/project_repo/data/external/AAPL_flat.csv")
AMZN_df = pd.read_csv("/Users/fahad/project_repo/data/external/AMZN_flat.csv")

In [61]:
#vertically stack the MSFT and AAPL dataframes on top of eachother
df = pd.concat([MSFT_df, AAPL_df], axis=0)
#vertically stack the MSFT&AAPL df and the AMZN dataframe ontop of eachother
df = pd.concat([df,AMZN_df],axis = 0)

y = df.Label # define the target variable (dependent variable) as y

df = df.drop(['Date','Label'], axis =1)


In [62]:
#define the target variable as y 
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3)
#print (X_train.head(5))
print(X_train.shape[1])
print(X_train.shape[0])

360
541


# Model

In [85]:
def make_timeseries_classifier(sample_num,window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=16):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.
    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).
    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int filter_length: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int nb_filter: The number of different filters to learn (roughly, input patterns to recognize).
    """
    model = Sequential((
        # The first conv layer learns `nb_filter` filters (aka kernels), each of size ``(filter_length, nb_input_series)``.
        # Its output will have shape (None, window_size - filter_length + 1, nb_filter), i.e., for each position in
        # the input timeseries, the activation of each filter at that position.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu', input_shape=(window_size,nb_input_series)),
        MaxPooling1D(),     # Downsample the output of convolution by 2X.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='sigmoid'),     # For binary classification, change the activation to 'sigmoid'
    ))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # To perform (binary) classification instead:
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    
    return model

In [86]:

model_1 = make_timeseries_classifier(X_train.shape[0],X_train.shape[1],3)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(360, 1), filters=16, kernel_size=3)`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=16, kernel_size=3)`


In [87]:
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_25 (Conv1D)           (None, 358, 16)           64        
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 179, 16)           0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 177, 16)           784       
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, 88, 16)            0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 1408)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 1409      
Total params: 2,257
Trainable params: 2,257
Non-trainable params: 0
_________________________________________________________________


In [80]:
def evaluate_model(trainX, trainy, testX, testy,model):
    #sometimes batch size must be a divisor of rows
    verbose, epochs, batch_size = 1, 10, 32
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    
    return accuracy


In [91]:
model_1.fit(np.array(X_train).reshape((*X_train.shape,1)),np.array(y_train).reshape((*y_train.shape,1)),epochs=20,batch_size=1, verbose=1)

Epoch 1/20
541/541 [==============================] - 1s 1ms/step - loss: 0.4017 - acc: 0.8429
Epoch 2/20
541/541 [==============================] - 1s 1ms/step - loss: 0.2785 - acc: 0.8928
Epoch 3/20
541/541 [==============================] - 1s 1ms/step - loss: 0.2262 - acc: 0.9094
Epoch 4/20
541/541 [==============================] - 1s 1ms/step - loss: 0.2035 - acc: 0.9131
Epoch 5/20
541/541 [==============================] - 1s 1ms/step - loss: 0.1884 - acc: 0.9224
Epoch 6/20
541/541 [==============================] - 1s 1ms/step - loss: 0.1614 - acc: 0.9224
Epoch 7/20
541/541 [==============================] - 1s 1ms/step - loss: 0.1598 - acc: 0.9372
Epoch 8/20
541/541 [==============================] - 1s 1ms/step - loss: 0.1462 - acc: 0.9335
Epoch 9/20
541/541 [==============================] - 1s 1ms/step - loss: 0.1369 - acc: 0.9464
Epoch 10/20
541/541 [==============================] - 1s 1ms/step - loss: 0.1365 - acc: 0.9372
Epoch 11/20
541/541 [============================

In [82]:
X_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
23,0.468719,0.471194,0.473923,0.472880,0.026460,0.480051,0.530707,0.507642,0.536421,0.162060,...,0.506608,0.502931,0.027454,0.654104,0.551421,0.477261,0.509144,0.197641,0.0,0.0
182,0.019363,0.020412,0.019597,0.020482,0.135017,0.337802,0.467295,0.019374,0.018325,0.011924,...,0.019838,0.019439,0.079544,0.523487,0.482383,0.019934,0.019084,0.013919,0.0,0.0
207,0.014752,0.015711,0.015098,0.015733,0.022026,0.521980,0.233387,0.009060,0.007858,0.018869,...,0.016889,0.016866,0.025077,0.593370,0.236473,0.014543,0.010879,0.012098,0.0,0.0
210,0.014621,0.015826,0.014880,0.015922,0.040869,0.459413,0.236596,0.015448,0.015785,0.022562,...,0.010876,0.010886,0.012705,0.357955,0.234835,0.014724,0.014711,0.016287,0.0,0.0
231,0.021356,0.024041,0.021415,0.023068,0.107104,0.472226,0.467663,0.020150,0.017606,0.043923,...,0.027143,0.029643,0.092109,0.542929,0.467004,0.022363,0.019814,0.052748,0.0,0.0


In [ ]:
evaluate_model(trainX, trainy,testX,testy):
    verbose, epochs, batch_size = 1, 10, 32
    trainX.reshape(trainX[0], trainX[1], 1)
    n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], trainy.shape[0]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape = (trainX[1], 1)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
    accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy